In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load data from CSV file
data = pd.read_csv('/Users/mahinbindra/Downloads/Trees.csv')

In [3]:
# Create a new model
m = gp.Model("tree_planting")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [4]:
# Add variables
x = m.addVars(18, vtype=GRB.BINARY, name="x")  # Whether a location is chosen
t = m.addVars(18, vtype=GRB.INTEGER, name="t")  # Number of trees planted at each location

In [5]:
# Cost coefficients
fixed_costs = data['Cost Per Location']
planting_costs = data['Planting Cost Per Tree']

In [6]:
# Objective: minimize total costs
m.setObjective(gp.quicksum(fixed_costs[i] * x[i] + planting_costs[i] * t[i] for i in range(18)), GRB.MINIMIZE)

In [7]:
# Add constraints
# Tree planting constraints
m.addConstrs((103000 * x[i] <= t[i] for i in range(18)), "min_trees")
m.addConstrs((t[i] <= 970000 * x[i] for i in range(18)), "max_trees")

# Location selection constraints
m.addConstr(x[0] + x[1] + x[2] + x[3] <= 2, "group1_max2")
m.addConstr(x[5] + x[8] + x[11] + x[14] + x[17] == 3, "group2_exact3")
m.addConstr(x[1] + x[3] + x[5] + x[7] + x[11] + x[13] + x[15] + x[17] <= 4, "group3_max4")

# Updated exclusive location constraints
m.addConstr(x[4] <= 1 - x[5], "exclude5_from6")
m.addConstr(x[4] <= 1 - x[6], "exclude5_from7")
m.addConstr(x[4] <= 1 - x[7], "exclude5_from8")

# Updated dependent location constraints
m.addConstr(2 * x[8] <= x[12] + x[14] + x[16], "dependent9_on13_15_17")

# Balance between locations 1-9 and 10-18
m.addConstr(sum(t[i] for i in range(9)) == sum(t[i] for i in range(9, 18)), "balance_trees")

# Total number of trees planted
m.addConstr(sum(t[i] for i in range(18)) == 10000000, "total_trees")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 45 rows, 36 columns and 135 nonzeros
Model fingerprint: 0x1d33ea8b
Variable types: 0 continuous, 36 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e-01, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Presolve time: 0.01s
Presolved: 45 rows, 36 columns, 135 nonzeros
Variable types: 0 continuous, 36 integer (18 binary)

Root relaxation: objective 2.916598e+06, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2916597.94    0    3          - 2916597.94      -     -    0s
H    0     0                    3021000.0000 2974865.98  1.53%     -    0s
     0

In [9]:
# Print solution
if m.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    locations = m.getAttr('x', x)
    trees = m.getAttr('x', t)
    for i in range(18):
        if locations[i] > 0.5:  # Only print chosen locations
            print(f"Location {i+1}: Plant {trees[i]:.0f} trees")
else:
    print('No optimal solution found')

Optimal solution found:
Location 1: Plant 150001 trees
Location 3: Plant 969999 trees
Location 6: Plant 970000 trees
Location 7: Plant 970000 trees
Location 8: Plant 970000 trees
Location 9: Plant 970000 trees
Location 11: Plant 150000 trees
Location 13: Plant 970000 trees
Location 14: Plant 970000 trees
Location 15: Plant 970000 trees
Location 16: Plant 970000 trees
Location 17: Plant 970000 trees


In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load data from CSV file
data = pd.read_csv('/Users/mahinbindra/Downloads/Trees.csv')

# Create a new model
m = gp.Model("tree_planting_no_constraints")

# Add variables
x = m.addVars(18, vtype=GRB.BINARY, name="x")  # Whether a location is chosen
t = m.addVars(18, vtype=GRB.INTEGER, name="t")  # Number of trees planted at each location

# Cost coefficients
fixed_costs = data['Cost Per Location']
planting_costs = data['Planting Cost Per Tree']

# Objective: minimize total costs
m.setObjective(gp.quicksum(fixed_costs[i] * x[i] + planting_costs[i] * t[i] for i in range(18)), GRB.MINIMIZE)

# Add constraints
# Each location can plant up to 970,000 trees if chosen
m.addConstrs((t[i] <= 970000 * x[i] for i in range(18)), "max_trees")

# Total number of trees planted
m.addConstr(sum(t[i] for i in range(18)) == 10000000, "total_trees")

# Solve the model
m.optimize()

# Print solution
if m.status == GRB.OPTIMAL:
    print('Optimal solution found:')
    locations = m.getAttr('x', x)
    trees = m.getAttr('x', t)
    for i in range(18):
        if locations[i] > 0.5:  # Only print chosen locations
            print(f"Location {i+1}: Plant {trees[i]:.0f} trees")
else:
    print('No optimal solution found')


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 19 rows, 36 columns and 54 nonzeros
Model fingerprint: 0xe875a456
Variable types: 0 continuous, 36 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e-01, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+07, 1e+07]
Found heuristic solution: objective 3306500.0000
Presolve time: 0.00s
Presolved: 19 rows, 36 columns, 54 nonzeros
Variable types: 0 continuous, 36 integer (18 binary)
Found heuristic solution: objective 3306498.5500

Root relaxation: objective 2.609732e+06, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2609731.96    0    1 3306498.55 2609731.96  21.